In [2]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/top18minus2/top18_without_top2.csv
/kaggle/input/twitter-data/newData.csv
/kaggle/input/glove6b50dtxt/glove.6B.50d.txt


In [4]:
# df = pd.read_csv('../input/twitter-data/newData.csv')

In [5]:
# type(df['label_to_emoji'].value_counts()[:20].to_dict())

In [6]:
# top20Emoji_count = df['label_to_emoji'].value_counts()[:20].to_dict()

In [7]:
# top20Emoji = ['😂', '😭', '😩', '😅', '😔', '💀', '😊', '👀', '😌', '😁', '🙏', '😍', '😉', '👍',
#        '😒', '😳', '😐', '💯', '😎', '😆']

In [8]:
# top20Emoji_count

In [9]:
# newDf = df[df['label_to_emoji'].isin(top20Emoji[2:])]

In [10]:
# newDf = newDf.drop(columns='Unnamed: 0')

In [11]:
# newDf = newDf.reset_index(drop=True)

In [12]:
# newDf

In [13]:
# newDf['label_int'].value_counts().index

In [14]:
# mapping = {
#     53:0, 29:1, 43:2, 11:3, 34:4, 5:5, 36:6, 25:7, 63:8, 37:9, 33:10, 8:11, 42:12, 59:13, 40:14, 22:15, 38:16,
#     30:17
# }

In [15]:
# newDf['label_int'] = newDf['label_int'].map(mapping)

In [16]:
# newDf = newDf.drop(columns=['label_to_emoji','newLabel'])

In [17]:
# import nltk
# nltk.download('omw-1.4')
# import spacy
# from nltk.stem import WordNetLemmatizer

# nlp = spacy.load("en_core_web_sm")
# lemmatizer = WordNetLemmatizer()

# def tweet_preprocessing(tweet, nlp=nlp, lemmatizer=lemmatizer):
    
#     stopwords = nltk.corpus.stopwords.words('english')
#     newStopWords = ['url', 'email', 'percent', 'money', 'phone','user','time','date', 'number']
#     stopwords.extend(newStopWords)
#     stop_words = set(stopwords) - set(['not','no','nor'])
#     rm_stop_words_lemma = []
    
#     doc = nlp(tweet)
#     for token in doc:                                            # tokenization
#         if token.text not in stop_words and token.pos_!="PUNCT": 
#             lemma_word = lemmatizer.lemmatize(token.text)        # removing stop
#             rm_stop_words_lemma.append(lemma_word)               # lemmatizing words 
            
#     processed = " ".join(rm_stop_words_lemma)
#     return processed

In [18]:
# newDf["pre_punc"] = newDf["pre_punc"].apply(tweet_preprocessing)

In [19]:
# def removeNoise(text):
#     temp = text.split()
#     temp_list = []
#     for word in temp:
#         if len(word)>2:
#             temp_list.append(word)
#     return ' '.join(temp_list)

In [20]:
# newDf["pre_punc"] = newDf["pre_punc"].apply(removeNoise)

In [21]:
# newDf.head()

In [22]:
# newDf.to_csv('top18_without_top2.csv',index=False)

In [3]:
newDf = pd.read_csv('../input/top18minus2/top18_without_top2.csv')

In [6]:
newDf = newDf[newDf['label_int'].isin([0,1,2])]

In [7]:
newDf['label_int'].value_counts()

0    20430
1    17256
2    15225
Name: label_int, dtype: int64

In [8]:
newDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52911 entries, 1 to 207326
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   pre_punc   52906 non-null  object
 1   label_int  52911 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


In [9]:
newDf = newDf.dropna()
newDf=newDf.reset_index(drop=True)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
c_vect = CountVectorizer()
X = c_vect.fit_transform(newDf["pre_punc"])

In [11]:
from sklearn.model_selection import train_test_split
y = newDf["label_int"].values
X_train, X_test ,y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=2022,stratify=newDf['label_int'])

In [12]:
X_train

<44970x27349 sparse matrix of type '<class 'numpy.int64'>'
	with 316211 stored elements in Compressed Sparse Row format>

In [13]:
print("Training X Dim ",X_train.shape)
print("Training y Dim ",y_train.shape)

Training X Dim  (44970, 27349)
Training y Dim  (44970,)


In [14]:
from sklearn.naive_bayes import MultinomialNB
NB_classifier = MultinomialNB()
NB_classifier.fit(X_train, y_train)

MultinomialNB()

In [15]:
from sklearn.metrics import classification_report
y_predict_test = NB_classifier.predict(X_test)
print(classification_report(y_test, y_predict_test))

              precision    recall  f1-score   support

           0       0.52      0.65      0.58      3064
           1       0.52      0.50      0.51      2588
           2       0.48      0.34      0.40      2284

    accuracy                           0.51      7936
   macro avg       0.51      0.50      0.49      7936
weighted avg       0.51      0.51      0.50      7936



In [16]:
from sklearn.model_selection import train_test_split 
train_data, test_data = train_test_split(newDf, test_size = 0.3, stratify = newDf.label_int)

In [17]:
x_train = train_data['pre_punc'].values
y_train = train_data['label_int'].values
x_test = test_data['pre_punc'].values
y_test = test_data['label_int'].values

In [18]:
x_train.shape, x_test.shape

((37034,), (15872,))

In [19]:
f = open("../input/glove6b50dtxt/glove.6B.50d.txt", encoding='utf8')

In [20]:
embedding_index = {}

for line in f:
    values = line.split()
    word = values[0]
    emb = np.array(values[1:], dtype ='float')
    embedding_index[word] = emb

In [21]:
embedding_index['twitter'].shape

(50,)

In [22]:
def get_embedding_output(X):
    maxLen = 20
    embedding_output = np.zeros((len(X), maxLen, 50))
    
    for ix in range(X.shape[0]):
        my_example = X[ix].split()
        
        for ij in range(len(my_example)): 
            if (embedding_index.get(my_example[ij].lower()) is not None) and (ij<maxLen):
                embedding_output[ix][ij] = embedding_index[my_example[ij].lower()]
            
    return embedding_output

In [23]:
x_train_embed = get_embedding_output(x_train)
x_test_embed = get_embedding_output(x_test)

In [24]:
from tensorflow.keras.utils import to_categorical

In [25]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [26]:
from keras.models import Sequential 
from keras.layers import LSTM, Dense, Dropout, Bidirectional

In [27]:
model = Sequential()
model.add(Bidirectional(LSTM(units = 512, return_sequences=True), input_shape = (20,50)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(units=256)))
model.add(Dropout(0.3))
# model.add(Dense(units=128, activation='relu'))
# model.add(Dense(units=64, activation='relu'))
# model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=3, activation='softmax'))
model.summary()

2022-11-17 13:25:12.360655: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 13:25:12.442992: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 13:25:12.443840: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 13:25:12.445462: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 20, 1024)          2306048   
_________________________________________________________________
dropout (Dropout)            (None, 20, 1024)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               2623488   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 3)                 1539      
Total params: 4,931,075
Trainable params: 4,931,075
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics =['accuracy'])

In [29]:
hist = model.fit(x_train_embed, y_train, validation_split=0.2, shuffle=True, batch_size=64, epochs=25)

2022-11-17 13:25:25.017410: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25


2022-11-17 13:25:30.265983: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


463/463 [==============================] - 17s 22ms/step - loss: 1.0638 - accuracy: 0.4366 - val_loss: 1.0457 - val_accuracy: 0.4570
Epoch 2/25
463/463 [==============================] - 9s 20ms/step - loss: 1.0435 - accuracy: 0.4635 - val_loss: 1.0356 - val_accuracy: 0.4740
Epoch 3/25
463/463 [==============================] - 9s 20ms/step - loss: 1.0319 - accuracy: 0.4738 - val_loss: 1.0286 - val_accuracy: 0.4855
Epoch 4/25
463/463 [==============================] - 9s 20ms/step - loss: 1.0182 - accuracy: 0.4864 - val_loss: 1.0271 - val_accuracy: 0.4797
Epoch 5/25
463/463 [==============================] - 9s 20ms/step - loss: 1.0020 - accuracy: 0.5006 - val_loss: 1.0261 - val_accuracy: 0.4860
Epoch 6/25
463/463 [==============================] - 9s 20ms/step - loss: 0.9807 - accuracy: 0.5137 - val_loss: 1.0444 - val_accuracy: 0.4806
Epoch 7/25
463/463 [==============================] - 9s 20ms/step - loss: 0.9459 - accuracy: 0.5419 - val_loss: 1.0625 - val_accuracy: 0.4802
Epoch 8/2

KeyboardInterrupt: 

In [30]:
model.evaluate(x_test_embed, y_test)

496/496 [==============================] - 4s 6ms/step - loss: 1.4766 - accuracy: 0.4359


[1.4766395092010498, 0.43586188554763794]

In [31]:
y_pred = model.predict(x_test_embed)
from sklearn.metrics import classification_report
y_pred = np.array([np.argmax(pred) for pred in y_pred])
y_test = np.array([np.argmax(test) for test in y_test])
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.47      0.51      0.49      6128
           1       0.44      0.40      0.42      5177
           2       0.38      0.37      0.38      4567

    accuracy                           0.44     15872
   macro avg       0.43      0.43      0.43     15872
weighted avg       0.43      0.44      0.43     15872



In [ ]:
model.save('BiLSTM.h5')